In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Only necessary if visualization is needed
%%bash
git clone --depth 1 https://github.com/tensorflow/models TensorFlow/models
cd TensorFlow/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
pip install -U --pre tensorflow=="2.3.0"

Processing /content/TensorFlow/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1619135 sha256=d935595eeb833d78158b9aacd06c099393046334e7f09652bfbd887db44377be
  Stored in directory: /tmp/pip-ephem-wheel-cache-3r0t6wj1/wheels/1e/a6/ce/339480921228eb089bee0cdaf7efbaa7b5cf803ec6420ab905
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
  Using cached https://files.pythonhosted.org/packages/97/ae/0b08f53498417914f2274cc3b5576d2b83179b0cbb209457d0fde0152174/tensorflow-2.3.0-cp36-cp36m-manylinux2010_x86_64.whl
  Using ca

fatal: destination path 'TensorFlow/models' already exists and is not an empty directory.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tf-models-official 2.4.0 has requirement tensorflow>=2.4.0, but you'll have tensorflow 2.3.0 which is incompatible.


In [10]:
%matplotlib inline

import time
from pathlib import Path

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

IS_PRETRAINED = True

MODEL_NAME = "efficientdet_d1_coco17_tpu-32"

if IS_PRETRAINED:
     PATH_TO_SAVED_MODEL = str(Path("pre-trained-models") / MODEL_NAME / "saved_model")
else:
    PATH_TO_SAVED_MODEL = str(Path("exported-models") / MODEL_NAME / "saved_model")
PATH_TO_LABEL_MAP = "data/label_map.pbtxt"
DO_VISUALIZATION = True

# if not Path("exported-models").exists():
#   Path("exported-models").symlink_to(PATH_DRIVE_ROOT / "exported-models")
#   Path("data").symlink_to(PATH_DRIVE_ROOT / "data")
#   Path("outputs").symlink_to(PATH_DRIVE_ROOT / "outputs")

# 'val' or 'test'
INFER_MODE = 'val'
# RECORD_FILE = "data/512x512-nms-ratio/test.tfrecord"

In [2]:
print("Loading model...", end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...WARNING:tensorflow:Importing a function (__inference_EfficientDet-D1_layer_call_and_return_conditional_losses_119744) with ops with custom gradients. Will likely fail if a gradient is requested.
Done! Took 112.37074041366577 seconds


In [8]:
if DO_VISUALIZATION:
  from object_detection.utils import label_map_util
  from object_detection.utils import visualization_utils as viz_utils

  COCO_LABEL_MAP = Path(r"H:\Workspace\VinBigData\TensorFlow\models\research\object_detection\data\mscoco_label_map.pbtxt")

  category_index = label_map_util.create_category_index_from_labelmap(
      str(COCO_LABEL_MAP), use_display_name=True)

In [ ]:
val_features = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/object/bbox/xmax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/xmin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/class/label': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'image/object/class/text': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    'image/width': tf.io.FixedLenFeature([], tf.int64)
}

test_features = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    'image/width': tf.io.FixedLenFeature([], tf.int64)
}

def parse_record(serialized):
    if INFER_MODE == 'val':
        features = val_features
    else:
        features = test_features

    parsed_example = tf.io.parse_single_example(serialized=serialized,features=features)
    print(parsed_example["image/object/class/label"])
    image = tf.io.decode_image(parsed_example['image/encoded'])
    image_id = parsed_example['image/filename']

    if INFER_MODE == 'val':
        boxes = tf.stack([
            parsed_example['image/object/bbox/ymin'],
            parsed_example['image/object/bbox/xmin'],
            parsed_example['image/object/bbox/ymax'],
            parsed_example['image/object/bbox/xmax']]
        )
        boxes = tf.transpose(boxes)
        labels = parsed_example['image/object/class/label']
    else:
        boxes, labels = None, None

    return image, image_id, boxes, labels

In [26]:
dataset = tf.data.TFRecordDataset(filenames=[RECORD_FILE]) \
    .map(tf.autograph.experimental.do_not_convert(parse_record),
         num_parallel_calls=tf.data.experimental.AUTOTUNE) \
    .cache() \
    .prefetch(tf.data.experimental.AUTOTUNE)

entries = []

for input_tensor, image_id, boxes, labels in tqdm(dataset):
    # Repeat grayscale image 3 times to get RGB image
    input_tensor = tf.repeat(input_tensor, 3, axis=2)

    # Add batch dimension
    detections = detect_fn(input_tensor[tf.newaxis, ...])

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    for box, class_id, score in zip(detections['detection_boxes'], detections['detection_classes'], detections['detection_scores']):
        entries.append((np.concatenate((np.array([image_id], dtype=object), [class_id], box, [score]))))

    if DO_VISUALIZATION:
        image_np_with_detections = input_tensor.numpy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
              image_np_with_detections,
              detections['detection_boxes'],
              detections['detection_classes'],
              detections['detection_scores'],
              category_index,
              use_normalized_coordinates=True,
              max_boxes_to_draw=200,
              min_score_thresh=.10,
              agnostic_mode=False)

        plt.figure()
        plt.imshow(image_np_with_detections)
        print('Done')
        break

# plt.show()

NameError: name 'parse_record' is not defined

In [ ]:
output_df = pd.DataFrame(entries, columns=['image_id', 'class_id', 'y_min', 'x_min', 'y_max', 'x_max', 'score'])
output_df["image_id"] = output_df["image_id"].transform(lambda x : x.numpy()).str.decode('utf-8')
output_df.to_csv(f"outputs/{MODEL_NAME}.csv", index=False)

In [78]:
DATA_FOLDER = Path(r"data/samples/train/imgs")

IMG_ROOT = r"H:\Workspace\AICity21\track4\data\samples\train\imgs" + '\\'

entries = []

for img_path in tqdm(DATA_FOLDER.iterdir()):
    img = tf.keras.preprocessing.image.load_img(img_path)
    img_name = Path(img_path).name
    input_tensor = tf.convert_to_tensor(tf.keras.preprocessing.image.img_to_array(img), dtype=tf.uint8)

    height, width, _ = input_tensor.shape

    # Add batch dimension
    detections = detect_fn(input_tensor[tf.newaxis, ...])

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    for box, class_id, score in zip(detections['detection_boxes'], detections['detection_classes'], detections['detection_scores']):
        if class_id in (3, 6, 8) and score > 0.4:
            org_box = [box[1] * width, box[0] * height, box[3] * width, box[2] * height]
            entries.append((np.concatenate((np.array([IMG_ROOT + img_name], dtype=object), org_box, [score], [None]))))

    if False:
        image_np_with_detections = input_tensor.numpy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
              image_np_with_detections,
              detections['detection_boxes'],
              detections['detection_classes'],
              detections['detection_scores'],
              category_index,
              use_normalized_coordinates=True,
              max_boxes_to_draw=200,
              min_score_thresh=.40,
              agnostic_mode=False)

        plt.figure()
        plt.imshow(image_np_with_detections)
        plt.show()
        break

plt.show()

0it [00:00, ?it/s]

In [79]:
df = pd.DataFrame(entries)
df.head()
df.to_csv("data/samples/train/annotations.csv", header=False, index=False)